# Works on first System

In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait, Select
import time
import os
import getpass
from bs4 import BeautifulSoup

In [2]:
# Kill any existing Chrome instances to avoid conflicts
os.system("taskkill /im chrome.exe /f")
user = getpass.getuser() # Fetch current user's name
service = Service() # Set up ChromeDriver service
options = webdriver.ChromeOptions() # Define Chrome options
options.add_argument(f"--user-data-dir=C:\\Users\\{user}\\AppData\\Local\\Google\\Chrome\\User Data")
options.add_argument("--profile-directory=Default")

# Create a WebDriver instance
# driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)#,service=service)
driver = webdriver.Chrome(service=service, options=options)


The chromedriver version (99.0.4844.51) detected in PATH at C:\Users\12408\chromedriver.exe might not be compatible with the detected chrome version (126.0.6478.127); currently, chromedriver 126.0.6478.126 is recommended for chrome 126.*, so it is advised to delete the driver in PATH and retry


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 99
Current browser version is 126.0.6478.127 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	Ordinal0 [0x00E89943+2595139]
	Ordinal0 [0x00E1C9F1+2148849]
	Ordinal0 [0x00D14528+1066280]
	Ordinal0 [0x00D342C3+1196739]
	Ordinal0 [0x00D2FE37+1179191]
	Ordinal0 [0x00D2D6F1+1169137]
	Ordinal0 [0x00D60530+1377584]
	Ordinal0 [0x00D6018A+1376650]
	Ordinal0 [0x00D5B806+1357830]
	Ordinal0 [0x00D36086+1204358]
	Ordinal0 [0x00D36F96+1208214]
	GetHandleVerifier [0x0102B232+1658114]
	GetHandleVerifier [0x010E312C+2411516]
	GetHandleVerifier [0x00F1F261+560433]
	GetHandleVerifier [0x00F1E366+556598]
	Ordinal0 [0x00E2286B+2173035]
	Ordinal0 [0x00E275F8+2192888]
	Ordinal0 [0x00E276E5+2193125]
	Ordinal0 [0x00E311FC+2232828]
	BaseThreadInitThunk [0x769A7BA9+25]
	RtlInitializeExceptionChain [0x77BAC10B+107]
	RtlClearBits [0x77BAC08F+191]


In [4]:
SEARCH_TERM = "코로나" #(covid)
SEARCH_TERM_YEAR = "2020"

driver.get("https://www.facebook.com/mohwpr")
time.sleep(2)
# get page name
page_title = driver.find_element(By.TAG_NAME, "h1").text

# Find the 'more options' button by its unique identifier
more_options_button = driver.find_element(By.XPATH, "//div[@role='button'][@aria-label='See Options']")
more_options_button.click()

time.sleep(2)  # Wait a bit for the menu to appear

# Now, to click on the 'Search' option within the menu, you would locate it similarly
# This assumes 'Search' is a visible text option within the opened menu
search_option = driver.find_element(By.XPATH, "//span[text()='Search']")
search_option.click()
time.sleep(3) 

# After clicking the search option button...
# Use WebDriverWait to wait for the input to be clickable
wait = WebDriverWait(driver, 10)
search_input = wait.until(EC.element_to_be_clickable((By.XPATH, f'//input[@placeholder="Search in {page_title.strip()}\'s posts, photos and tags"]')))

# Now send the keys to the search input
search_input.send_keys(SEARCH_TERM)
search_input.send_keys(Keys.ENTER)
time.sleep(3) 

# Wait for the filter options to be available after the search
date_filter_dropdown = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, "//div[@role='combobox'][@aria-label='Filter by Date posted']"))
)
# Click the combobox to display the options
date_filter_dropdown.click()

# Wait for the dropdown options to be visible
option_xpath = f"//span[text()='{SEARCH_TERM_YEAR.strip()}']"
date_option = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, option_xpath))
)

# Click the desired date filter option
date_option.click()

time.sleep(5) 

NameError: name 'driver' is not defined

In [5]:
# Function to click 'See more'
def expand_see_more(driver):
    try:
        while True:
            see_more = driver.find_elements(By.XPATH, "//div[@role='button' and text()='See more']")
            if not see_more:
                break
            for button in see_more:
                driver.execute_script("arguments[0].scrollIntoView();", button)
                driver.execute_script("arguments[0].click();", button)
                time.sleep(1)
    except Exception as e:
        print("Error expanding 'See more':", e)


last_height = driver.execute_script("return document.body.scrollHeight")

post_content_recorded = []
results = []
# Scroll for 600 seconds
start_time = time.time()
end_time = start_time + 120
while time.time() < end_time:
    expand_see_more(driver)  # Expand all 'See more' in the current view
    # Extract post texts
    soup = BeautifulSoup(driver.page_source, "html.parser")
    posts = soup.find_all("div", class_="x1iorvi4 x1pi30zi x1l90r2v x1swvt13")

    for post in posts:
        post_and_comments = {}
        post_content = post.text
        if "… See more" not in post_content and post_content not in post_content_recorded:
            post_and_comments["post"] = post_content
            post_content_recorded.append(post_content)
            results.append(post_and_comments)

    
    # Scroll down
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    # Check if the end of the page was reached
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height       
    


NameError: name 'driver' is not defined

In [3]:
results

NameError: name 'results' is not defined

In [59]:
driver.quit()

# Works on second System

In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait, Select
import time
import os
import getpass
from bs4 import BeautifulSoup

In [47]:
# Setup WebDriver
# Kill any existing Chrome instances to avoid conflicts
os.system("taskkill /im chrome.exe /f")
user = getpass.getuser() # Fetch current user's name
service = Service() # Set up ChromeDriver service
options = webdriver.ChromeOptions() # Define Chrome options
options.add_argument(f"--user-data-dir=C:\\Users\\{user}\\AppData\\Local\\Google\\Chrome\\User Data")
options.add_argument("--profile-directory=Default")

# Create a WebDriver instance
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver = webdriver.Chrome(service=service, options=options) # comment this if it raises error and uncomment the line above

SEARCH_TERM = "코로나" #(covid)
SEARCH_TERM_YEAR = "2020"


driver.get("https://www.facebook.com/mohwpr")
time.sleep(10)

from selenium.webdriver.common.action_chains import ActionChains

# Create an instance of ActionChains
actions = ActionChains(driver)

# Move to a location on the page and click (e.g., at (10, 10) coordinates)
actions.move_by_offset(10, 10).click().perform()
time.sleep(3)  

# get page name
page_title = driver.find_element(By.TAG_NAME, "h1").text

# Find the 'more options' button by its unique identifier
more_options_button = driver.find_element(By.XPATH, "//div[@role='button'][@aria-label='See options']")
more_options_button.click()

time.sleep(2)  # Wait a bit for the menu to appear

# Now, to click on the 'Search' option within the menu, you would locate it similarly
# This assumes 'Search' is a visible text option within the opened menu
search_option = driver.find_element(By.XPATH, "//span[text()='Search']")
search_option.click()
time.sleep(3) 

# After clicking the search option button...
# Use WebDriverWait to wait for the input to be clickable
wait = WebDriverWait(driver, 10)
search_input = wait.until(EC.element_to_be_clickable((By.XPATH, f'//input[@placeholder="Search in {page_title.strip()}\'s posts, photos, and tags"]')))

# Now send the keys to the search input
search_input.send_keys(SEARCH_TERM)
search_input.send_keys(Keys.ENTER)
time.sleep(3) 




In [48]:
# Wait for the filter options to be available after the search
date_filter_dropdown = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, "//div[@role='combobox'][@aria-label= 'Filter by Date Posted']"))
)
# Click the combobox to display the options
date_filter_dropdown.click()

# Wait for the dropdown options to be visible
option_xpath = f"//span[text()='{SEARCH_TERM_YEAR.strip()}']"
date_option = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, option_xpath))
)

# Click the desired date filter option
date_option.click()

time.sleep(5) 


In [50]:

# Function to click 'See more'
def expand_see_more(driver):
    try:
        while True:
            see_more = driver.find_elements(By.XPATH, "//div[@role='button' and text()='See more']")
            if not see_more:
                break
            for button in see_more:
                driver.execute_script("arguments[0].scrollIntoView();", button)
                driver.execute_script("arguments[0].click();", button)
                time.sleep(1)
    except Exception as e:
        print("Error expanding 'See more':", e)


last_height = driver.execute_script("return document.body.scrollHeight")

post_content_recorded = []
results = []
# Scroll for 600 seconds
start_time = time.time()
end_time = start_time + 120
while time.time() < end_time:
    expand_see_more(driver)  # Expand all 'See more' in the current view
    # Extract post texts
    soup = BeautifulSoup(driver.page_source, "html.parser")
    posts = soup.find_all("div", class_="x1iorvi4 x1pi30zi x1l90r2v x1swvt13")

    for post in posts:
        post_and_comments = {}
        post_content = post.text
        if "… See more" not in post_content and post_content not in post_content_recorded:
            post_and_comments["post"] = post_content
            post_content_recorded.append(post_content)
            results.append(post_and_comments)

    # Scroll down
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    # Check if the end of the page was reached
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height       
    
